In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta

In [23]:
faults = pd.read_csv("../data/J1939Faults.csv", low_memory = False)
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [25]:
pd.set_option('display.max_columns', None)

In [27]:
faults_prepped = faults.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [29]:
exclude_coords = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.7  # Radius of Earth in km - roughly 3958.7 miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2 # this is the haversine formula that's accounting for the curvature of the earth. think calculus & a polar coordinate system.
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)) # we're calculating the central angle between the two points
    return R * c # calculates the arc length (distance) we're multiplying the radius of earth and the central angle
threshold = 0.5  # Distance in km - this is roughly about 3.1 miles
distances = np.array([
    haversine(faults_prepped['Latitude'], faults_prepped['Longitude'], lat, lon)
    for lat, lon in exclude_coords
]) # I'm calculating the difference between the excluded coordinates and geo coordinate in my dataframe. the array is contains the distances as scalar quantities (floats).
# which rows are outside all vicinities
outside_vicinity = (distances > threshold).all(axis=0) # boolean array true if its outside vicinity
faults_prepped = faults_prepped[outside_vicinity]

In [31]:
diagnostics_pivot = diagnostics.pivot(index=['FaultId'], columns='Name', values='Value')

In [33]:
data = diagnostics_pivot.merge(faults_prepped, left_on = 'FaultId', right_on = 'RecordID', how = 'inner')

In [35]:
columns_to_change = ['AcceleratorPedal', 
                     'BarometricPressure',
                     'CruiseControlSetSpeed',
                     'DistanceLtd',
                     'EngineCoolantTemperature',
                     'EngineLoad',
                     'EngineOilPressure',
                     'EngineOilTemperature',
                     'EngineRpm',
                     'EngineTimeLtd',
                     'FuelLevel',
                     'FuelLtd',
                     'FuelRate',
                     'FuelTemperature',
                     'IntakeManifoldTemperature',
                     'Speed',
                     'SwitchedBatteryVoltage',
                     'TurboBoostPressure',
                    ]

In [37]:
for column in columns_to_change:
    data[column] = data[column].str.replace(',', '.').astype(np.float64)

In [39]:
data['LocationTimeStamp'] = pd.to_datetime(data['LocationTimeStamp'])
data['EventTimeStamp'] = pd.to_datetime(data['EventTimeStamp'])

In [41]:
data = data.drop(['ecuModel','ecuMake', 'MCTNumber', 'ServiceDistance'], axis=1)

In [43]:
data['target_time_min'] = data.loc[data['spn'] == 5246]['EventTimeStamp'] - pd.Timedelta(hours=2) 
data['target_time_max'] = data.loc[data['spn'] == 5246]['EventTimeStamp'] - pd.Timedelta(hours=3)

start_times = data['target_time_max'].dropna()
end_times = data['target_time_min'].dropna()

In [111]:
data.loc[data['EventTimeStamp'].between(start_times.iloc[1], end_times.iloc[1])]['EventTimeStamp']

2339   2015-02-23 12:55:38
2340   2015-02-23 12:56:11
2341   2015-02-23 12:57:13
2342   2015-02-23 12:58:49
2343   2015-02-23 12:59:52
               ...        
2428   2015-02-23 13:26:00
2430   2015-02-23 13:43:50
2435   2015-02-23 13:31:06
2437   2015-02-23 13:31:29
2635   2015-02-23 13:08:41
Name: EventTimeStamp, Length: 91, dtype: datetime64[ns]

In [47]:
event_series = []
idx = 0

for idx in range(len(start_times)):
    event = data.loc[data['EventTimeStamp'].between(start_times.iloc[idx], end_times.iloc[idx])]['EventTimeStamp']
    idx = idx + 1
    event_series.append(event)

In [97]:
series = pd.Series(event_series)
series = series.apply(pd.Series)
series = series.stack()
series = series.reset_index(drop=True)

In [101]:
series

0       2015-02-23 02:08:40
1       2015-02-23 02:10:09
2       2015-02-23 02:10:54
3       2015-02-23 02:10:56
4       2015-02-23 02:15:12
                ...        
38533   2020-02-17 14:28:33
38534   2020-02-17 14:28:33
38535   2020-02-17 14:35:16
38536   2020-02-17 14:37:30
38537   2020-02-17 15:06:55
Length: 38538, dtype: datetime64[ns]

In [105]:
data['target'] = data['EventTimeStamp'].isin(series)

In [113]:
data.loc[data['target']]

,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,target_time_min,target_time_max,target
1729,0.0,14.7175,False,64.62260,416696.20,161.6,8.0,26.68,174.31250,693.125,7982.90,NaN,46375.667901,0.686850,NaN,True,93.2,1023,True,0.000000,3276.75,NaN,0.58,2011,1009900,2015-02-23 02:08:40,Low (Severity Low) Engine Coolant Level,unknown,unknown,0,111,17,True,1,1450,41.287824,-87.292916,2015-02-23 02:09:17,NaT,NaT,True
1730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,2012,1009903,2015-02-23 02:10:09,Low (Severity Low) Engine Coolant Level,unknown,unknown,0,111,17,False,1,1450,41.287731,-87.292962,2015-02-23 02:10:05,NaT,NaT,True
1731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,2013,1009907,2015-02-23 02:10:54,Abnormal Update Rate Steering Wheel Angle,unknown,unknown,11,1807,9,False,127,1432,40.534259,-76.431990,2015-02-23 02:10:49,NaT,NaT,True
1732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,2014,1009912,2015-02-23 02:10:56,Abnormal Update Rate Steering Wheel Angle,unknown,unknown,11,1807,9,True,127,1432,40.534259,-76.431990,2015-02-23 02:11:31,NaT,NaT,True
1734,0.0,14.5725,False,66.48672,239164.60,55.4,21.0,46.40,53.54375,699.625,4642.60,NaN,33448.804794,1.294447,32.0,True,41.0,18431,True,0.000000,3276.75,0,0.29,2016,1009928,2015-02-23 02:15:12,Low (Severity Low) Engine Coolant Level,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,0,111,17,True,24,1630,40.352222,-76.727870,2015-02-23 02:15:48,NaT,NaT,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053066,0.0,13.7750,False,66.48672,391674.40,185.0,11.0,21.46,208.90630,597.625,7771.35,46.8,53485.462422,0.528346,NaN,True,114.8,1279,False,0.000000,NaN,100,0.58,1242748,121377830,2020-02-17 14:28:33,High Voltage (Fuel Level),NaN,NaN,49,96,3,True,126,1851,36.317962,-82.341990,2020-02-17 14:29:09,NaT,NaT,True
1053067,0.0,13.7750,False,66.48672,391674.40,185.0,11.0,21.46,208.90630,597.625,7771.35,46.8,53485.462422,0.528346,NaN,True,114.8,1279,False,0.000000,NaN,100,0.58,1242749,121377831,2020-02-17 14:28:33,High Voltage (Left Fuel Level Sensor),NaN,NaN,49,829,3,True,126,1851,36.317962,-82.341990,2020-02-17 14:29:09,NaT,NaT,True
1053068,21.6,14.6450,False,0.00000,50444.68,177.8,49.0,35.96,188.82500,839.125,1010.65,56.4,6883.927417,2.456808,NaN,True,127.4,50175,False,2.417522,NaN,100,2.61,1242750,121379562,2020-02-17 14:35:16,Low (Severity Medium) Transmission Air Tank Pr...,NaN,NaN,3,37,18,True,11,2342,28.545000,-82.387222,2020-02-17 14:35:53,NaT,NaT,True
1053069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50175,NaN,NaN,NaN,NaN,NaN,1242751,121379707,2020-02-17 14:37:30,Low (Severity Medium) Transmission Air Tank Pr...,NaN,NaN,3,37,18,False,11,2342,28.544907,-82.387268,2020-02-17 14:37:26,NaT,NaT,True


In [63]:
data.loc[data['spn'] == 5246]

,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,target_time_min,target_time_max,target
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4645.45,NaN,33470.466902,NaN,NaN,False,NaN,22527,NaN,NaN,NaN,NaN,NaN,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,0,5246,0,True,1,1630,40.733009,-74.087777,2015-02-23 05:08:23,2015-02-23 03:05:44,2015-02-23 02:05:44,False
2589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,0,5246,0,True,1,1487,28.077361,-81.897083,2015-02-23 15:54:58,2015-02-23 13:54:22,2015-02-23 12:54:22,False
5141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,5714,1070646,2015-02-25 13:53:08,NaN,unknown,unknown,0,5246,0,True,1,1329,39.399583,-82.974768,2015-02-25 13:56:31,2015-02-25 11:53:08,2015-02-25 10:53:08,False
5236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,5810,1071907,2015-02-25 14:47:00,NaN,unknown,unknown,0,5246,0,False,1,1329,39.399629,-82.974814,2015-02-25 14:46:56,2015-02-25 12:47:00,2015-02-25 11:47:00,False
5949,NaN,14.5000,False,64.62260,441699.6,185.0,10.0,20.30,198.16250,648.125,9087.95,57.2,69605.769379,0.634015,NaN,True,140.0,22527,NaN,0.0,3276.75,NaN,0.58,6535,1097942,2015-02-26 22:24:29,NaN,04993120*00021657*082113134117*07700053*I0*BBZ*,79466573,0,5246,0,True,1,1419,37.596805,-85.865555,2015-02-26 22:25:05,2015-02-26 20:24:29,2015-02-26 19:24:29,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,1239564,119961467,2020-02-06 08:03:09,NaN,04358814*06005963*051718174436*09401683*G1*BDR*,79897320,0,5246,0,False,1,1854,35.943472,-83.823240,2020-02-06 08:03:05,2020-02-06 06:03:09,2020-02-06 05:03:09,False
1052278,0.0,13.9925,False,57.78752,477486.8,197.6,10.0,21.46,204.96880,599.500,9344.30,72.4,63966.356499,0.462303,NaN,True,120.2,22527,True,0.0,NaN,100,0.58,1241841,120905759,2020-02-13 13:32:39,NaN,04358814*06026985*051718174436*09401683*G1*BDR*,79903054,0,5246,0,True,1,1872,35.707268,-81.397037,2020-02-13 13:33:15,2020-02-13 11:32:39,2020-02-13 10:32:39,False
1052292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,1241858,120910417,2020-02-13 14:01:40,NaN,04358814*06026985*051718174436*09401683*G1*BDR*,79903054,0,5246,0,False,1,1872,35.708101,-81.395648,2020-02-13 13:59:51,2020-02-13 12:01:40,2020-02-13 11:01:40,False
1052528,0.0,14.7900,False,0.00000,334736.2,17.6,51.0,84.68,18.10625,651.500,11554.20,100.0,393.220099,2.892693,19.4,True,26.6,255,True,0.0,NaN,0,0.58,1242137,121038018,2020-02-14 11:21:54,NaN,NaN,NaN,49,5246,19,True,88,302,38.349490,-85.708425,2020-02-14 11:22:30,2020-02-14 09:21:54,2020-02-14 08:21:54,False


In [117]:
data.to_csv('../data/data_clean.csv')

In [ ]:
data_clean = pd.read_csv('../data/data_clean.csv')

In [119]:
data_clean.groupby('EquipmentID')['target']

AttributeError: 'NoneType' object has no attribute 'groupby'